![ibm cloud logo](./images/ibm-cloud.png)

# Case Study - Data ingestion

The goal of this case study is to put into practice the important concepts from module 1.  We will go through the basic process that begins with refining the business opportunity and ensuring that it is articulated using a scientific thought process.

The business opportunity and case study was first mentioned in Unit 2 of module 1 and like the AAVIAL company itself these data were created for learning purposes.  We will be using the AAVAIL example as a basis for this case study.  [Watch the video again](https://vimeo.com/348708995) if you need a refresher.  You will be gathering data from several provided sources, staging it for quality assurance and saving it in a target destination that is most appropriate.

Watch the video to review the important concepts from the units you just covered and to see an overview of the objectives for this case study.

In [1]:
from IPython.display import IFrame
IFrame('https://player.vimeo.com/video/354996550', width=600,height=400)

## Case study overall objectives

1. Gather all relevant data from the sources of provided data
2. Implement several checks for quality assurance 
3. Take the initial steps towards automation of the ingestion pipeline

## Getting started

Download this notebook and open it locally using a Jupyter server. Alternatively you may use Watson Studio.  To make using Watson Studio easier we have provided a zip archive file containing the files needed to complete this case study in Watson Studio. See the [Getting started with Watson Studio](m1-u5-5-watson-studio.rst) page.

**You will need the following files to complete this case study**

* [m1-u6-case-study.ipynb](m1-u6-case-study.ipynb)
* [m1-u6-case-study-solution.ipynb](./notebooks/m1-u6-case-study-solution.ipynb)
* [aavail-customers.db](./data/aavail-customers.db)
* [aavail-steams.csv](./data/aavail-streams.csv)

1. Fill in all of the places in this notebook marked with ***YOUR CODE HERE*** or ***YOUR ANSWER HERE***
2. When you have finished the case study there will be a short quiz

You may review the rest of this content as part of the notebook, but once you are ready to get started be ensure that you are working with a *live* version either as part of Watson Studio or locally.

## Make Notebook Run in Watson Studio

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
# START CODE BLOCK
# cos2file - takes an object from Cloud Object Storage and writes it to file on container file system.
# Uses the IBM project_lib library.
# See https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/project-lib-python.html
# Arguments:
# p: project object defined in project token
# data_path: the directory to write the file
# filename: name of the file in COS

import os
def cos2file(p,data_path,filename):
    data_dir = p.project_context.home + data_path
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    open( data_dir + '/' + filename, 'wb').write(p.get_file(filename).read())

# file2cos - takes file on container file system and writes it to an object in Cloud Object Storage.
# Uses the IBM project_lib library.
# See https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/project-lib-python.html
# Arguments:
# p: prooject object defined in project token
# data_path: the directory to read the file from
# filename: name of the file on container file system

import os
def file2cos(p,data_path,filename):
    data_dir = p.project_context.home + data_path
    path_to_file = data_dir + '/' + filename
    if os.path.exists(path_to_file):
        file_object = open(path_to_file, 'rb')
        p.save_data(filename, file_object, set_project_asset=True, overwrite=True)
    else:
        print("file2cos error: File not found")
# END CODE BLOCK

### Create Data Directory

In [4]:
cos2file(project, '/data', 'aavail-customers.db')
cos2file(project, '/data', 'aavail-streams.csv')

## Data Sources

The data you will be sourcing from is contained in two sources.

1. A database ([SQLite](https://www.sqlite.org/index.html)) of `customer` data
2. A [CSV file](https://en.wikipedia.org/wiki/Comma-separated_values) of `stream` level data

   >You will create a simple data pipeline that
   (1) simplifies the data for future analysis
   (2) performs quality assurance checks.

The process of building *the data ingestion pipeline* entails extracting data, transforming it, and loading it into an appropriate data storage technology.  When constructing a pipeline it is important to keep in mind that they generally process data in batches.  For example, data may be compiled during the day and the batch could be processed during the night.  The data pipeline may also be optimized to execute as a streaming computation (i.e., every event is handled as it occurs).

## PART 1: Gathering the data

The following is an [Entity Relationship Diagram (ERD)](https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model) that details the tables and contents of the database.

<img src="./images/aavail-schema.svg" alt="customer database schema" style="width: 600px;"/>

In [5]:
## all the imports you will need for this case study
import os
import pandas as pd
import numpy as np
import sqlite3

Much of the data exist in a database.  You can connect to is using the `sqlite3` Python package with the function shown below.  Note that is is good practice to wrap your connect functions in a [try-except statement](https://docs.python.org/3/tutorial/errors.html) to cleanly handle exceptions.

In [6]:
def connect_db(file_path):
    try:
        conn = sqlite3.connect(file_path)
        print("...successfully connected to db\n")
    except Error as e:
        print("...unsuccessful connection\n",e)
    
    return(conn)

In [7]:
## make the connection to the database
conn = connect_db('../data/aavail-customers.db')

## print the table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';")]
print(tables)

...successfully connected to db

['CUSTOMER', 'INVOICE', 'INVOICE_ITEM', 'COUNTRY']


### QUESTION 1:

**extract the relevant data from the DB**

Query the database and extract the following data into a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).
 
* Customer ID (integer)
* Last name
* First name
* DOB
* City
* State
* Country (the name NOT the country_id)
* Gender

Remember that that SQL is case-insensitive, but it is traditional to use ALL CAPS for SQL keywords. It is also a convention to end SQL statements with a semi-colon.  

#### Resources

* [W3 schools SQL tutorial](https://www.w3schools.com/sql)
* [W3 schools SQL joins](https://www.w3schools.com/sql/sql_join.asp)

In [8]:
query = """SELECT 
            CUSTOMER.customer_id, 
            CUSTOMER.last_name,
            CUSTOMER.first_name,
            CUSTOMER.DOB,
            CUSTOMER.city, 
            CUSTOMER.state, 
            COUNTRY.country_name, 
            CUSTOMER.gender
            FROM CUSTOMER
            LEFT JOIN COUNTRY 
            ON CUSTOMER.country_id = COUNTRY.country_id;"""

df_customers = pd.DataFrame([row for row in conn.execute(query)], 
                            columns=["customer_id", "last_name", "first_name", "DOB", "city", "state", "country_name", "gender"])
df_customers.head()

,customer_id,last_name,first_name,DOB,city,state,country_name,gender
0,1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m
1,2,Garza,Ensley,04/12/89,singapore,None,singapore,f
2,3,Carey,Lillian,09/12/97,Auburn,Alabama,united_states,f
3,4,Christensen,Beau,01/28/99,Hempstead,New York,united_states,m
4,5,Gibson,Ernesto,03/23/98,singapore,None,singapore,m


In [9]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 8 columns):
customer_id     1007 non-null int64
last_name       1007 non-null object
first_name      1007 non-null object
DOB             1007 non-null object
city            1007 non-null object
state           703 non-null object
country_name    1007 non-null object
gender          1007 non-null object
dtypes: int64(1), object(7)
memory usage: 63.0+ KB


The customers dataset has features type `int64` and `object`. Feature `state` contains also `NaN`.

### QUESTION 2:

**Extract the relevant data from the CSV file**

For each ```customer_id``` determine if a customer has stopped their subscription or not and save it in a dictionary or another data container.

In [10]:
df_streams = pd.read_csv('../data/aavail-streams.csv')
df_streams.head()

,customer_id,stream_id,date,subscription_stopped
0,1,1356,2018-12-01,0
1,1,1540,2018-12-04,0
2,1,1395,2018-12-11,0
3,1,1255,2018-12-22,0
4,1,1697,2018-12-23,0


In [11]:
df_streams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19032 entries, 0 to 19031
Data columns (total 4 columns):
customer_id             19032 non-null int64
stream_id               19032 non-null int64
date                    19032 non-null object
subscription_stopped    19032 non-null int64
dtypes: int64(3), object(1)
memory usage: 594.8+ KB


The streams dataset doesn't contain any `NaN` values.

In [12]:
df_streams.describe()

,customer_id,stream_id,subscription_stopped
count,19032.000000,19032.000000,19032.000000
mean,500.478142,1387.988703,0.014765
std,290.447934,224.176814,0.120613
min,1.000000,1000.000000,0.000000
25%,248.000000,1195.000000,0.000000
50%,497.000000,1389.000000,0.000000
75%,753.000000,1584.000000,0.000000
max,1000.000000,1776.000000,1.000000


The feature `subscription_stopped` is of type `int64` where the max value is 1 and the min is 0. I assume 1 represents those `streams_id` per `customer_id` that were stopped.

In [13]:
df_streams[df_streams.customer_id==3]

,customer_id,stream_id,date,subscription_stopped
37,3,1689,2019-01-24,0
38,3,1414,2019-01-27,0
39,3,1157,2019-01-27,0
40,3,1401,2019-02-12,0
41,3,1374,2019-02-13,0
42,3,1143,2019-02-19,0
43,3,1540,2019-02-27,0
44,3,1548,2019-03-01,0
45,3,1455,2019-03-09,0
46,3,1170,2019-03-17,0


For each customer I get the latest date and check whether the customer stopped the subscription or not.

In [14]:
df_churns = df_streams.sort_values(["customer_id","date"], ascending=False).groupby(["customer_id"]).head(1)
df_churns = df_churns.sort_values(["customer_id"]).reset_index(drop=True)
df_churns["is_subscriber"] = np.where(df_churns.subscription_stopped == 1, 0, 1)
df_churns.head()

,customer_id,stream_id,date,subscription_stopped,is_subscriber
0,1,1521,2019-03-12,0,1
1,2,1497,2019-06-07,1,0
2,3,1689,2019-05-30,0,1
3,4,1140,2018-03-28,0,1
4,5,1280,2019-05-20,0,1


In [15]:
df_churns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
customer_id             1000 non-null int64
stream_id               1000 non-null int64
date                    1000 non-null object
subscription_stopped    1000 non-null int64
is_subscriber           1000 non-null int64
dtypes: int64(4), object(1)
memory usage: 39.1+ KB


In [16]:
df_churns.describe()

,customer_id,stream_id,subscription_stopped,is_subscriber
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,1379.847000,0.245000,0.755000
std,288.819436,224.048319,0.430302,0.430302
min,1.000000,1000.000000,0.000000,0.000000
25%,250.750000,1187.750000,0.000000,1.000000
50%,500.500000,1363.000000,0.000000,1.000000
75%,750.250000,1577.000000,0.000000,1.000000
max,1000.000000,1776.000000,1.000000,1.000000


## PART 2: Checks for quality assurance

Sometimes it is known in advance which types of data integrity issues to expect, but other times it is during the Exploratory Data Analysis (EDA) process that these issues are identified.  After extracting data it is important to include checks for quality assurance even on the first pass through the AI workflow.  Here you will combine the data into a single structure and provide a couple checks for quality assurance.

### QUESTION 3: 

**Implement checks for quality assurance**

1. Remove any repeat customers based on ```customer_id```
2. Remove stream data that do not have an associated ```stream_id```
3. Check for missing values

**1. Remove any repeat customers based on `customer_id`**

In [17]:
is_duplicate = df_customers.duplicated(subset=["customer_id"])
print("number of duplicates:", len(df_customers[is_duplicate]))
df_customers[is_duplicate]

number of duplicates: 7


,customer_id,last_name,first_name,DOB,city,state,country_name,gender
1000,1,Todd,Kasen,07/30/98,Rock Hill,South Carolina,united_states,m
1001,11,Rivas,Rohan,01/23/05,Pawtucket,Rhode Island,united_states,m
1002,21,Hickman,Claudia,06/08/81,singapore,None,singapore,f
1003,31,Rubio,Dalary,07/12/00,Indio,California,united_states,f
1004,301,Fisher,Jamie,12/31/82,singapore,None,singapore,m
1005,401,Hill,Brodie,11/30/98,singapore,None,singapore,m
1006,801,Snow,Kora,07/23/70,singapore,None,singapore,f


In [18]:
# remove the duplicate records from the customer dataset:
df_customers = df_customers[~is_duplicate].reset_index(drop=True)

In [19]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
customer_id     1000 non-null int64
last_name       1000 non-null object
first_name      1000 non-null object
DOB             1000 non-null object
city            1000 non-null object
state           700 non-null object
country_name    1000 non-null object
gender          1000 non-null object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB


**2. Remove stream data that do not have an associated `stream_id`**

In [20]:
print("number of records that do not have an associated stream id:", df_streams["stream_id"].isna().sum())

is_na = df_streams["stream_id"].isna()
df_streams = df_streams[~is_na].reset_index(drop=True)

number of records that do not have an associated stream id: 0


In [21]:
df_streams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19032 entries, 0 to 19031
Data columns (total 4 columns):
customer_id             19032 non-null int64
stream_id               19032 non-null int64
date                    19032 non-null object
subscription_stopped    19032 non-null int64
dtypes: int64(3), object(1)
memory usage: 594.8+ KB


**3. Check for missing values**

In [22]:
# missing data in customers dataset
total = df_customers.isnull().sum().sort_values(ascending=False)
percent = (df_customers.isnull().sum()/df_customers.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
state,300,0.3
gender,0,0.0
country_name,0,0.0
city,0,0.0
DOB,0,0.0
first_name,0,0.0
last_name,0,0.0
customer_id,0,0.0


In [23]:
df_customers[df_customers.isna().any(1)].head()

,customer_id,last_name,first_name,DOB,city,state,country_name,gender
1,2,Garza,Ensley,04/12/89,singapore,None,singapore,f
4,5,Gibson,Ernesto,03/23/98,singapore,None,singapore,m
6,7,Tate,Daxton,12/23/70,singapore,None,singapore,m
11,12,Ross,Romina,05/05/80,singapore,None,singapore,f
17,18,Torres,Zayne,10/11/74,singapore,None,singapore,m


In [24]:
# missing data in streams dataset
total = df_streams.isnull().sum().sort_values(ascending=False)
percent = (df_streams.isnull().sum()/df_streams.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
subscription_stopped,0,0.0
date,0,0.0
stream_id,0,0.0
customer_id,0,0.0


The stream dataset doesn't contain any NaN values.

In [25]:
df_streams[df_streams.isna().any(1)].head()

,customer_id,stream_id,date,subscription_stopped


### QUESTION 4: 

**combine the data into a single data structure**

For this example, the two most convenient structures for this task are Pandas dataframes and NumPy arrays.  At a minimum ensure that your structure accommodates the following.

1. A column for `customer_id`
2. A column for `country`
3. A column for ```age``` that is created from ```DOB```
4. A column ```customer_name``` that is created from ```first_name``` and ```last_name```
5. A column to indicate churn called ```is_subscriber```
7. A column that indicates ```subscriber_type``` that comes from ```invoice_item```
6. A column to indicate the total ```num_streams```

#### Resources

* [Python's datetime library](https://docs.python.org/3/library/datetime.html)
* [NumPy's datetime data type](https://docs.scipy.org/doc/numpy/reference/arrays.datetime.html)


In [26]:
from datetime import datetime

df_clean = df_customers.copy()

df_clean["date_of_birth"] = pd.to_datetime(df_customers['DOB'], format="%m/%d/%y")
df_clean.loc[df_clean['date_of_birth'].dt.year >= 2020, 'date_of_birth'] -= pd.DateOffset(years=100)
df_clean["age"] = datetime.now().year - df_clean.date_of_birth.dt.year
df_clean["customer_name"] = df_clean.first_name + " " + df_clean.last_name
df_clean.drop(["last_name", "first_name", "DOB", "city", "state", "gender", "date_of_birth"], axis=1, inplace=True)
df_clean.head()

,customer_id,country_name,age,customer_name
0,1,united_states,22,Kasen Todd
1,2,singapore,31,Ensley Garza
2,3,united_states,23,Lillian Carey
3,4,united_states,21,Beau Christensen
4,5,singapore,22,Ernesto Gibson


In [27]:
df_clean = df_clean.merge(df_churns[["customer_id", "is_subscriber"]], how="inner", on="customer_id")
df_clean.head()

,customer_id,country_name,age,customer_name,is_subscriber
0,1,united_states,22,Kasen Todd,1
1,2,singapore,31,Ensley Garza,0
2,3,united_states,23,Lillian Carey,1
3,4,united_states,21,Beau Christensen,1
4,5,singapore,22,Ernesto Gibson,1


In [28]:
query = """SELECT 
            INVOICE.customer_id, 
            INVOICE.invoice_item_id,
            INVOICE_ITEM.invoice_item
            FROM INVOICE
            INNER JOIN INVOICE_ITEM
            ON INVOICE.invoice_item_id = INVOICE_ITEM.invoice_item_id;"""

df_invoices = pd.DataFrame([row for row in conn.execute(query)], columns=["customer_id", "invoice_item_id", "invoice_item"])

In [29]:
df_invoices.duplicated(subset=["customer_id"]).any()

False

`customer_id` doesn't have any duplicates in the invoice dataset.

In [30]:
df_clean = df_clean.merge(df_invoices[["customer_id", "invoice_item"]], how="inner", on="customer_id")
df_clean.rename(columns={"invoice_item":"subscriber_type"}, inplace=True)
df_clean.head()

,customer_id,country_name,age,customer_name,is_subscriber,subscriber_type
0,1,united_states,22,Kasen Todd,1,aavail_premium
1,2,singapore,31,Ensley Garza,0,aavail_unlimited
2,3,united_states,23,Lillian Carey,1,aavail_premium
3,4,united_states,21,Beau Christensen,1,aavail_basic
4,5,singapore,22,Ernesto Gibson,1,aavail_premium


In [31]:
df_clean = df_clean.merge(df_streams.groupby(["customer_id"])["stream_id"].count().reset_index(), how="inner", on="customer_id")
df_clean.rename(columns={"stream_id":"num_streams"}, inplace=True)
df_clean.head()

,customer_id,country_name,age,customer_name,is_subscriber,subscriber_type,num_streams
0,1,united_states,22,Kasen Todd,1,aavail_premium,21
1,2,singapore,31,Ensley Garza,0,aavail_unlimited,16
2,3,united_states,23,Lillian Carey,1,aavail_premium,25
3,4,united_states,21,Beau Christensen,1,aavail_basic,18
4,5,singapore,22,Ernesto Gibson,1,aavail_premium,21


In [32]:
df_clean.describe()

,customer_id,age,is_subscriber,num_streams
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,28.034000,0.755000,19.032000
std,288.819436,9.114599,0.430302,4.840497
min,1.000000,14.000000,0.000000,1.000000
25%,250.750000,22.000000,1.000000,17.000000
50%,500.500000,24.000000,1.000000,20.000000
75%,750.250000,34.000000,1.000000,22.000000
max,1000.000000,57.000000,1.000000,30.000000


## PART 3: Automating the process

To ensure that you code can be used to automate this process.  First you will save you dataframe or numpy array as a CSV file.  

### QUESTION 5:

**Take the initial steps towards automation**

1. Save your cleaned, combined data as a CSV file.
2. From the code above create a function or class that performs all of the steps given a database file and a streams CSV file.
3. Run the function in batches and write a check to ensure you got the same result that you did in the code above.

There will be some logic involved to ensure that you do not write the same data twice to the target CSV file.

Shown below is some code that will split your streams file into two batches. 

**1. Save your cleaned, combined data as a CSV file.**

In [33]:
df_clean.to_csv('../data/aavail-clean.csv', index=False)

file2cos(project, '/data', 'aavail-clean.csv')

In [34]:
## code to split the streams csv into batches
df_all = pd.read_csv('../data/aavail-streams.csv')
half = int(round(df_all.shape[0] * 0.5))
df_part1 = df_all[:half]
df_part2 = df_all[half:]
df_part1.to_csv('../data/aavail-streams-1.csv', index=False)
df_part2.to_csv('../data/aavail-streams-2.csv', index=False)

**2. From the code above create a function or class that performs all of the steps given a database file and a streams CSV file.**

Create a script directory to store the scripts for production

In [35]:
def create_script_dir(p, script_path):
    script_dir = p.project_context.home + script_path
    if not os.path.exists(script_dir):
        print("...create script directory")
        os.makedirs(script_dir)
    else:
        print("...script directory exists")
        
create_script_dir(project, "/scripts")

...create script directory


You will need to save your function as a file.  The following cell demonstrates how to do this from within a notebook. 

You will also need to be able to pass the file names to your function without hardcoding them into the script itself.  This is an important step towards automation.  Here are the two libraries commonly used to accomplish this in Python.

* [getopt](https://docs.python.org/3/library/getopt.html)
* [argparse](https://docs.python.org/3/library/argparse.html)

In [36]:
%%writefile ../scripts/aavail-data-ingestor.py
#!/usr/bin/env python

import os
import sys
import getopt
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime


DATA_DIR = os.path.join("..","data")

def connect_db(file_path):
    """
    function to connect to aavail database
    """
    try:
        conn = sqlite3.connect(file_path)
        print("...successfully connected to db\n")
    except Error as e:
        print("...unsuccessful connection\n",e)
    
    return(conn)

def ingest_db_data(conn):
    """
    load and clean database data
    """
    query = """SELECT 
            CUSTOMER.customer_id, 
            CUSTOMER.last_name,
            CUSTOMER.first_name,
            CUSTOMER.DOB,
            CUSTOMER.city, 
            CUSTOMER.state, 
            COUNTRY.country_name, 
            CUSTOMER.gender
            FROM CUSTOMER
            LEFT JOIN COUNTRY 
            ON CUSTOMER.country_id = COUNTRY.country_id;"""
    
    # execute query and store the results in a pandas dataframe
    _data = [row for row in conn.execute(query)]
    columns=["customer_id", "last_name", "first_name", "DOB", "city", "state", "country_name", "gender"]
    df_customers = pd.DataFrame(_data, columns=columns)
    print("...imported db dataset of {} rows and {} columns".format(df_customers.shape[0], df_customers.shape[1]))
    
    # implement checks for quality assurance
    is_duplicate = df_customers.duplicated(subset=["customer_id"])
    print("...removed {} number of duplicates customer ids".format(len(df_customers[is_duplicate])))
    df_customers = df_customers[~is_duplicate].reset_index(drop=True)
    
    return df_customers

def ingest_stream_data(file_path):
    """
    load and clean stream data
    """
    # read streams data
    df_streams = pd.read_csv(file_path)
    print("...imported streams dataset of {} rows and {} columns".format(df_streams.shape[0], df_streams.shape[1]))
    
    # determine customer churn from streams dataset
    df_churns = df_streams.sort_values(["customer_id","date"], ascending=False).groupby(["customer_id"]).head(1)
    df_churns = df_churns.sort_values(["customer_id"]).reset_index(drop=True)
    df_churns["is_subscriber"] = np.where(df_churns.subscription_stopped == 1, 0, 1)
    
    # implement checks for quality assurance
    print("...removed {} missing stream ids".format(df_streams["stream_id"].isna().sum()))
    is_na = df_streams["stream_id"].isna()
    df_streams = df_streams[~is_na].reset_index(drop=True)
    
    return df_streams, df_churns

def process_dataframes(df_customers, df_streams, df_churns, conn):
    """
    combine the data into a single data structure
    """
    
    # create a clean copy of the customers dataframe and add new attributes
    df_clean = df_customers.copy()
    df_clean["date_of_birth"] = pd.to_datetime(df_customers['DOB'], format="%m/%d/%y")
    df_clean.loc[df_clean['date_of_birth'].dt.year >= 2020, 'date_of_birth'] -= pd.DateOffset(years=100)
    df_clean["age"] = datetime.now().year - df_clean.date_of_birth.dt.year
    df_clean["customer_name"] = df_clean.first_name + " " + df_clean.last_name
    df_clean.drop(["last_name", "first_name", "DOB", "city", "state", "gender", "date_of_birth"], axis=1, inplace=True)
    
    # ensure we are working with correctly ordered customer_ids df_customers
    if not np.array_equal(df_clean['customer_id'], df_customers['customer_id']):
        raise Exception("indexes are out of order or unmatched---needs to fix")
        
    df_clean = df_clean.merge(df_churns[["customer_id", "is_subscriber"]], how="inner", on="customer_id")
        
    # query the db to create a invoice item map
    query = """SELECT 
            INVOICE.customer_id, 
            INVOICE.invoice_item_id,
            INVOICE_ITEM.invoice_item
            FROM INVOICE
            INNER JOIN INVOICE_ITEM
            ON INVOICE.invoice_item_id = INVOICE_ITEM.invoice_item_id;"""
    
    df_invoices = pd.DataFrame([row for row in conn.execute(query)], columns=["customer_id", "invoice_item_id", "invoice_item"])
    
    # implement checks for quality assurance
    is_duplicate = df_invoices.duplicated(subset=["customer_id"])
    if True in is_duplicate:
        df_invoices = df_invoices[~is_duplicate].reset_index(drop=True)
        
    # add new attributes (subscriber_type, num_streams)
    df_clean = df_clean.merge(df_invoices[["customer_id", "invoice_item"]], how="inner", on="customer_id")
    df_clean.rename(columns={"invoice_item":"subscriber_type"}, inplace=True)
    
    df_clean = df_clean.merge(df_streams.groupby(["customer_id"])["stream_id"].count().reset_index(), how="inner", on="customer_id")
    df_clean.rename(columns={"stream_id":"num_streams"}, inplace=True)
    
    return df_clean
        

def update_target(target_file, df_clean, overwrite=False):
    """
    update line by line in case data are large
    """
    
    if overwrite or not os.path.exists(target_file):
        df_clean.to_csv(target_file, index=False)   
    else:
        df_target = pd.read_csv(target_file)
        df_target.to_csv(target_file, mode='a', index=False)
        
if __name__ == "__main__":
  
    ## collect args
    arg_string = "%s -d db_filepath -s streams_filepath"%sys.argv[0]
    try:
        optlist, args = getopt.getopt(sys.argv[1:], 'd:s:')
    except getopt.GetoptError:
        print(getopt.GetoptError)
        raise Exception(arg_string)

    ## handle args
    streams_file = None
    db_file = None
    for o, a in optlist:
        if o == '-d':
            db_file = a
        if o == '-s':
            streams_file = a
            
    streams_file = os.path.join(DATA_DIR, streams_file)
    db_file = os.path.join(DATA_DIR, db_file)
    target_file = os.path.join(DATA_DIR, "aavail-target.csv")
    
    ## make the connection to the database
    conn = connect_db(db_file)

    ## ingest data base data
    df_customers = ingest_db_data(conn)
    df_streams, df_churn = ingest_stream_data(streams_file)
    df_clean = process_dataframes(df_customers, df_streams, df_churn, conn)
    
    ## write
    update_target(target_file, df_clean, overwrite=False)
    print("done")
    

Writing ../scripts/aavail-data-ingestor.py


Save the production script in the IBM database

In [37]:
file2cos(project, '/scripts', 'aavail-data-ingestor.py')

You may run the script you just created from the commandline directly or from within this notebook using:

In [38]:
%run ../scripts/aavail-data-ingestor.py -d aavail-customers.db -s aavail-streams.csv

...successfully connected to db

...imported db dataset of 1007 rows and 8 columns
...removed 7 number of duplicates customer ids
...imported streams dataset of 19032 rows and 4 columns
...removed 0 missing stream ids
done


In [39]:
!rm ../data/aavail-target.csv
!python ../scripts/aavail-data-ingestor.py -d aavail-customers.db -s aavail-streams.csv
!wc -l ../data/aavail-target.csv

...successfully connected to db

...imported db dataset of 1007 rows and 8 columns
...removed 7 number of duplicates customer ids
...imported streams dataset of 19032 rows and 4 columns
...removed 0 missing stream ids
done
1001 ../data/aavail-target.csv


In [40]:
df_clean_all = pd.read_csv("../data/aavail-target.csv")
df_clean_all.head()

,customer_id,country_name,age,customer_name,is_subscriber,subscriber_type,num_streams
0,1,united_states,22,Kasen Todd,1,aavail_premium,21
1,2,singapore,31,Ensley Garza,0,aavail_unlimited,16
2,3,united_states,23,Lillian Carey,1,aavail_premium,25
3,4,united_states,21,Beau Christensen,1,aavail_basic,18
4,5,singapore,22,Ernesto Gibson,1,aavail_premium,21


Run the script once for each batch that you created and then load both the original and batch versions back into the notebook to check that they are the same. 

In [41]:
!rm ../data/aavail-target.csv
!python ../scripts/aavail-data-ingestor.py -d aavail-customers.db -s aavail-streams-1.csv
!wc -l ../data/aavail-target.csv

...successfully connected to db

...imported db dataset of 1007 rows and 8 columns
...removed 7 number of duplicates customer ids
...imported streams dataset of 9516 rows and 4 columns
...removed 0 missing stream ids
done
498 ../data/aavail-target.csv


In [42]:
df_clean_b1 = pd.read_csv("../data/aavail-target.csv")
df_clean_b1.head()

,customer_id,country_name,age,customer_name,is_subscriber,subscriber_type,num_streams
0,1,united_states,22,Kasen Todd,1,aavail_premium,21
1,2,singapore,31,Ensley Garza,0,aavail_unlimited,16
2,3,united_states,23,Lillian Carey,1,aavail_premium,25
3,4,united_states,21,Beau Christensen,1,aavail_basic,18
4,5,singapore,22,Ernesto Gibson,1,aavail_premium,21


In [43]:
!rm ../data/aavail-target.csv
!python ../scripts/aavail-data-ingestor.py -d aavail-customers.db -s aavail-streams-2.csv
!wc -l ../data/aavail-target.csv

...successfully connected to db

...imported db dataset of 1007 rows and 8 columns
...removed 7 number of duplicates customer ids
...imported streams dataset of 9516 rows and 4 columns
...removed 0 missing stream ids
done
505 ../data/aavail-target.csv


In [44]:
df_clean_b2 = pd.read_csv("../data/aavail-target.csv")
df_clean_b2.head()

,customer_id,country_name,age,customer_name,is_subscriber,subscriber_type,num_streams
0,497,united_states,17,Deacon Porter,1,aavail_unlimited,7
1,498,singapore,18,Romeo Campbell,0,aavail_basic,1
2,499,united_states,28,Eloise Hendricks,0,aavail_premium,6
3,500,united_states,21,Hayley Harrell,1,aavail_unlimited,21
4,501,united_states,23,Madelynn Conley,1,aavail_premium,4


### QUESTION 6:

**How can you improve the process?**

In paragraph form or using bullets write down some of the ways that you could improve this pipeline.

YOUR ANSWER HERE




* How do we ensure that the batches after concatenate do not generate duplicates?
* Batches should be created in the final, combined, cleaned dataset.
* I would add also some logical tests to assure the quality of the dataset, for example data type
* Kudos to the automation/productionize part
* Quality controls added to ensure that the DOB has been successfully converted
